In [1]:
import pandas as pd

In [2]:
import chardet

In [3]:
nombre_archivo = 'precios_semanas_20200419_20200426.xlsx'
# este nombre tiene que tomarse de la ejecución del script; para desarrollarlo lo hardcodeamos
extension = nombre_archivo.split('.')[1]
# guarda la extension del archivo
ruta = './Datasets/'
# harcodeamos ruta relativa
codfecha = (nombre_archivo.split('.')[0])[-8:]
fecha_carga = codfecha[0:4]+'/'+codfecha[4:6]+'/'+codfecha[6:8]

In [4]:
#archivo = open(str(ruta)+str(nombre_archivo), 'rb')
#rawdata=archivo.read()
#archivo.close()

In [5]:
#encod=chardet.detect(rawdata)['encoding']

In [6]:
dfprecio = pd.read_excel(str(ruta)+str(nombre_archivo), sheet_name=0)

In [7]:
dfprecio.head()

,precio,sucursal_id,producto_id
0,399.0,2-1-092,2288.0
1,299.0,2-1-206,2288.0
2,399.0,2-2-241,2288.0
3,49999.0,9-1-430,205870.0
4,53999.0,9-2-4,205870.0


In [8]:
dfprecio.describe(include='all')
#veo que hay precios muy elevados

,precio,sucursal_id,producto_id
count,4.771730e+05,478909,4.653900e+05
unique,NaN,222,NaN
top,NaN,49-1-2,NaN
freq,NaN,15083,NaN
mean,1.698493e+12,NaN,7.489733e+12
std,2.505227e+14,NaN,1.345933e+12
min,6.000000e+00,NaN,2.288000e+03
25%,8.590000e+02,NaN,7.790158e+12
50%,1.730000e+03,NaN,7.791290e+12
75%,4.730000e+03,NaN,7.794001e+12


In [9]:
def sacar_espacios(df):
    '''
    Quitar espacios vacíos de de ambos lados de las cols con strings.
    '''
    sacar_espacios = lambda x: x.strip() if isinstance(x, str) else x
    return df.applymap(sacar_espacios)
sacar_espacios(dfprecio)

,precio,sucursal_id,producto_id
0,399.0,2-1-092,2.288000e+03
1,299.0,2-1-206,2.288000e+03
2,399.0,2-2-241,2.288000e+03
3,49999.0,9-1-430,2.058700e+05
4,53999.0,9-2-4,2.058700e+05
...,...,...,...
478904,13999.0,25-1-1,9.569753e+12
478905,3499.0,25-1-1,9.795403e+12
478906,3125.0,5-1-3,9.990386e+12
478907,3125.0,5-1-3,9.990386e+12


In [10]:
dfprecio['precio']=dfprecio['precio'].astype(float)

In [11]:
dfprecio[dfprecio['precio'].isnull()|dfprecio['producto_id'].isnull()|dfprecio['sucursal_id'].isnull()]

,precio,sucursal_id,producto_id
10277,NaN,65-1-328,6.431315e+11
10673,NaN,65-1-328,6.552577e+11
10674,NaN,65-1-328,6.552577e+11
11434,3999.0,1-1-7,NaN
11435,5599.0,1-1-7,NaN
...,...,...,...
478738,4090.0,9-3-5222,NaN
478739,5090.0,9-3-5277,NaN
478740,11499.0,9-3-5227,NaN
478741,2190.0,9-3-5222,NaN


In [12]:
dfprecio.dropna(inplace=True)

In [13]:
dfprecio.info()
#observo que no hay mas nulos

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463753 entries, 0 to 478908
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   precio       463753 non-null  float64
 1   sucursal_id  463753 non-null  object 
 2   producto_id  463753 non-null  float64
dtypes: float64(2), object(1)
memory usage: 14.2+ MB


In [14]:
# comencemos con la columna producto_id
# creo una columna 'cant' que indique la cantidad de caracteres
dfprecio['cant']=dfprecio['producto_id'].apply(lambda x: len(str(x)))

In [15]:
dfprecio['cant'].unique()

array([ 6,  8,  9, 10, 11, 12, 13, 14, 15], dtype=int64)

In [16]:
#ejemplo 
dfprecio[dfprecio['cant']==6]

,precio,sucursal_id,producto_id,cant
0,399.0,2-1-092,2288.0,6
1,299.0,2-1-206,2288.0,6
2,399.0,2-2-241,2288.0,6


In [17]:




dfprecio['id2']=dfprecio['producto_id'].apply(lambda x: str(x)[-13:])
import re
dfprecio['caractid2']=dfprecio['id2'].apply(lambda x: re.findall('\D',x))


In [18]:
dfprecio['caractid2'].describe()
#observo que los unicos caracteres no numericos son '.' y '-'

count     463753
unique         1
top          [.]
freq      463753
Name: caractid2, dtype: object

In [19]:
#creo una funcion que normalice los 3 tipos de dato identificados.
# integers que terminan con .0 (eliminar el '.0' final)
# floats en notacion cientifica (eliminar el '.')
# sucursal id mezclado con cod producto (tomar el 3er termino del conjunto nn-nnnn-nnnnnn)

def prod_id (x):
    #str(x)
    if x[-2:]== '.0':
        x = x[:-2]
        return prod_id(x)
    elif '.' in x:
        x = x.replace('.','')
        return prod_id(x)
    elif '-' in x:
        a=0
        i=len(x)-1
        while a==0:
            if x[i]=='-':
                a=i
            i-=1
        x=x[a+1:]
        return prod_id(x)

    else:
        return x


In [20]:
dfprecio['producto_id']=dfprecio['producto_id'].astype(str)

In [21]:
dfprecio['producto_id']=dfprecio['producto_id'].apply(prod_id)

In [22]:


#vuelvo a chequear caracteres no numericos en la col transformada

dfprecio['id2']=dfprecio['producto_id'].apply(lambda x: str(x)[-13:])
dfprecio['caractid2']=dfprecio['id2'].apply(lambda x: re.findall('\D',x))


In [23]:
dfprecio['caractid2'].describe()
#observo que no hay caract no numericos

count     463753
unique         1
top           []
freq      463753
Name: caractid2, dtype: object

In [24]:

dfprecio.pop('cant')
dfprecio.pop('id2')


0                  2288
1                  2288
2                  2288
3                205870
4                205870
              ...      
478904    9569753142128
478905    9795403001143
478906    9990385651922
478907    9990385651939
478908    9990385651946
Name: id2, Length: 463753, dtype: object

In [25]:

dfprecio.pop('caractid2')


0         []
1         []
2         []
3         []
4         []
          ..
478904    []
478905    []
478906    []
478907    []
478908    []
Name: caractid2, Length: 463753, dtype: object

In [26]:

dfprecio.head()

,precio,sucursal_id,producto_id
0,399.0,2-1-092,2288
1,299.0,2-1-206,2288
2,399.0,2-2-241,2288
3,49999.0,9-1-430,205870
4,53999.0,9-2-4,205870


In [27]:
#normalizo sucursal
dfprecio_aux=dfprecio['sucursal_id'].str.split('-', expand=True)

In [28]:
dfprecio_aux[0]=dfprecio_aux[0].astype(int)
dfprecio_aux[1]=dfprecio_aux[1].astype(int)
dfprecio_aux[2]=dfprecio_aux[2].astype(int)


In [29]:
dfprecio_aux.head()
dfprecio_aux.describe()

,0,1,2
count,463753.000000,463753.000000,463753.000000
mean,15.982842,1.648229,577.194869
std,13.804133,1.152297,1174.780127
min,1.000000,1.000000,1.000000
25%,9.000000,1.000000,8.000000
50%,10.000000,1.000000,141.000000
75%,16.000000,2.000000,702.000000
max,50.000000,6.000000,6269.000000


In [30]:
dfprecio_aux['sucursal_id']=dfprecio_aux[0].astype(str)+'-'+dfprecio_aux[1].astype(str)+'-'+dfprecio_aux[2].astype(str)

In [31]:
dfprecio_aux.head()

,0,1,2,sucursal_id
0,2,1,92,2-1-92
1,2,1,206,2-1-206
2,2,2,241,2-2-241
3,9,1,430,9-1-430
4,9,2,4,9-2-4


In [32]:
dfprecio['sucursal_id']=dfprecio_aux['sucursal_id']

In [33]:
dfprecio['fecha']=fecha_carga

In [34]:
dfprecio.head()

,precio,sucursal_id,producto_id,fecha
0,399.0,2-1-92,2288,2020/04/26
1,299.0,2-1-206,2288,2020/04/26
2,399.0,2-2-241,2288,2020/04/26
3,49999.0,9-1-430,205870,2020/04/26
4,53999.0,9-2-4,205870,2020/04/26


In [35]:
dfprecio.rename(columns={'producto_id': 'Id_Producto', 'sucursal_id': 'Id_Sucursal'}, inplace=True)

In [36]:
dfprecio.insert(0, 'Id', 0)

In [37]:
#exportacion a csv de la tabla
#from encodings import utf_8


ruta_export = './Data_out/'
nombre_archivo_out = 'producto_precio_' + str(codfecha) + '.csv'
dfprecio.to_csv(ruta_export+nombre_archivo_out, index=False, encoding='utf-8-sig')

In [38]:
# dfprecio[dfprecio['precio']>= 100000]